Item 45: Consider @property Insterad of Refactoring Attributes

In [1]:
from datetime import datetime, timedelta

In [5]:
class Bucket:
    def __init__(self, period):
        self.period_delta = timedelta(seconds=period)
        self.reset_time = datetime.now()
        self.max_quota = 0
        self.quota_consumed = 0
    
    def __repr__(self):
        return(f'Bucket(max_quota={self.max_quota}, '
                    f'quota_consumed={self.quota_consumed})')
    
    @property
    def quota(self):
        return self.max_quota - self.quota_consumed
    
    @quota.setter
    def quota(self, amount):
        delta = self.max_quota - amount
        if amount == 0:
            # Quota being reset for a new period
            self.quota_consumed = 0
            self.max_quota = 0
        elif delta < 0:
            # Quota being filled for the new period
            assert self.quota_consumed == 0
            self.max_quota = amount
        else:
            # Quota being consumed during the period
            assert self.max_quota >= self.quota_consumed
            self.quota_consumed += delta


def fill(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        bucket.quota = 0
        bucket.reset_time = now
    bucket.quota += amount
    

def deduct(bucket, amount):
    now = datetime.now()
    if (now - bucket.reset_time) > bucket.period_delta:
        return False # Bucket hasn't been filled this period
    if bucket.quota - amount < 0:
        return False # Bucket was filled, but not enough
    
    bucket.quota -= amount
    return True

In [22]:
bucket = Bucket(60)
print('Initial', bucket)
fill(bucket, 100)
print('Filled', bucket)

if deduct(bucket, 99):
    print('Had 99 quota')
else:
    print('Not enough for 99 quota')
print('Now ', bucket)

if deduct(bucket, 3):
    print('Had 3 quota')
else:
    print('Not enought for 3 quota')
print('Still ', bucket)

Initial Bucket(max_quota=0, quota_consumed=0)
Filled Bucket(max_quota=100, quota_consumed=0)
Had 99 quota
Now  Bucket(max_quota=100, quota_consumed=99)
Not enought for 3 quota
Still  Bucket(max_quota=100, quota_consumed=99)


In [23]:
fill(bucket, 200)

AssertionError: 

In [8]:
import time

In [24]:
time.sleep(60)
print('quota: ', bucket.quota)
deduct(bucket, 1)

quota:  1


False

In [25]:
deduct(bucket, 0)

False